In [ ]:
# Github: https://github.com/swatichauhan08/uber_reviews/blob/main/code.py
# Streanlit has been checked and it is working

#### Q1. Pre-processing: Examine the dataset. ID the columns of interest. Drop special characters, html junk etc. Perform any other preprocessing and text-cleaning activity you think fits this context.

In [2]:
## setup chunk
import time   # to time 'em opns
t0 = time.time()    # start timer
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import csv
import string
from nltk.corpus import stopwords
# import mpld3  # conda install -c conda-forge mpld3 
t1 = time.time()

time.taken = round(t1-t0, 3)
print(time.taken)
print("\n")    # print newline

1.562




In [11]:
# importing the csv file into the dataframe
datafile = pd.read_csv(r'//Users/Dell/Desktop/FP1 Project - All Files/Final_Applicants_list.csv')
Candidate_Resume = pd.DataFrame(datafile, columns= ['Category','Resume','Exp (Years)','Name','Current Salary','Expected Salary','Offered Salary','Age','Current Location','Offered Location','Offer Status'])
print(Candidate_Resume)

                    Category  \
0            DevOps Engineer   
1               Data Science   
2               Data Science   
3               Data Science   
4               Data Science   
...                      ...   
1243             MS Dynamics   
1244     Java Cloud Services   
1245    Azure Cloud Services   
1246  Azure Devops Architect   
1247             MS Dynamics   

                                                 Resume  Exp (Years)  \
0     technical skills key skills ms technology net ...          NaN   
1     education details  mca   ymcaust  faridabad  h...          NaN   
2     skills c basics iot python matlab data science...          NaN   
3     skills  r  python  sap hana  tableau  sap hana...          NaN   
4     skills  python  tableau  data visualization  r...          NaN   
...                                                 ...          ...   
1243  core competencies  ant  maven  git  bitbucket ...          NaN   
1244  technical skills key skills ms te

In [12]:
#dropping unnecessary columns
Candidate_Resume.drop('Offer Status', axis=1, inplace=True)

In [13]:
#converting the data into lowercase for analysis
Candidate_Resume['Category'] = Candidate_Resume['Category'].str.lower()
Candidate_Resume['Resume'] = Candidate_Resume['Resume'].str.lower()
Candidate_Resume

,Category,Resume,Exp (Years),Name,Current Salary,Expected Salary,Offered Salary,Age,Current Location,Offered Location
0,devops engineer,technical skills key skills ms technology net ...,NaN,Avinash Joshi,2650000,3090000,2960000,43,Pune,Pune
1,data science,education details mca ymcaust faridabad h...,NaN,Karishma Neving,684000,1240000,1100000,28,Hyderabad,Hyderabad
2,data science,skills c basics iot python matlab data science...,NaN,Aaron Miles,603000,1120000,1100000,25,Gurgaon,Gurgaon
3,data science,skills r python sap hana tableau sap hana...,NaN,Dhruv Jain,745000,1180000,1080000,25,Bangalore,Bangalore
4,data science,skills python tableau data visualization r...,NaN,Grace Mallory,762000,1270000,1140000,26,Gurgaon,Gurgaon
...,...,...,...,...,...,...,...,...,...,...
1243,ms dynamics,core competencies ant maven git bitbucket ...,NaN,Kaviyarasan G,1100000,1700000,1700000,35,Chennai,Chennai
1244,java cloud services,technical skills key skills ms technology net ...,NaN,Divya Surishetty,985000,1200000,1200000,31,Hyderabad,Hyderabad
1245,azure cloud services,core skills project program management agil...,NaN,Madhu Sudhan,1260000,1700000,1700000,27,Hyderabad,Hyderabad
1246,azure devops architect,core skills project program management agil...,NaN,Padmaja Samantara,2525000,3200000,3200000,31,Bangalore,Bangalore


In [91]:
# creating the function for punctuation removal
def remove_punctuations(text):
    for char in string.punctuation:
        text = text.replace(char, '')
    return text

uber_reviews['Title'] = uber_reviews['Title'].apply(remove_punctuations)
uber_reviews['Review'] = uber_reviews['Review'].apply(remove_punctuations)
uber_reviews

,Title,Rating,Review,Date
0,dishonest and disgusting,1,for half an hour we tried every uber service t...,29-12-2020 01:14
1,free offer,2,if im not eligible for the offer stop floodin...,01-01-2021 23:17
2,inaccurate,2,consistently inaccurate uber eats eta and the ...,15-01-2021 23:38
3,bad,1,i had my rides canceled back to back they then...,08-12-2020 01:01
4,double charged me for an order,1,two of the same orders was added by accident t...,15-12-2020 04:02
...,...,...,...,...
485,uber,5,perdí mi cuenta no la puedo recuperar la use e...,16-01-2021 02:39
486,crap crap crap,1,still the same i was forced to use it in colom...,23-12-2020 00:15
487,sleeping drivers,1,it is a 30 minute commute from my household to...,16-12-2020 19:10
488,bad design re offer code redemption and issue ...,1,was sent a 30 off uber eats i thought about pl...,25-11-2020 23:06


In [92]:
# working with the corpus to remove the stopwords

stop = stopwords.words('english')
guided_list = ['im','uber','10','20','30']
stop_extended = stop + guided_list
uber_reviews['Title_nostop'] = uber_reviews['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_extended]))
uber_reviews['Review_nostop'] = uber_reviews['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_extended]))
uber_reviews

,Title,Rating,Review,Date,Title_nostop,Review_nostop
0,dishonest and disgusting,1,for half an hour we tried every uber service t...,29-12-2020 01:14,dishonest disgusting,half hour tried every service take short trip ...
1,free offer,2,if im not eligible for the offer stop floodin...,01-01-2021 23:17,free offer,eligible offer stop flooding email false infor...
2,inaccurate,2,consistently inaccurate uber eats eta and the ...,15-01-2021 23:38,inaccurate,consistently inaccurate eats eta food menus of...
3,bad,1,i had my rides canceled back to back they then...,08-12-2020 01:01,bad,rides canceled back back still charged account...
4,double charged me for an order,1,two of the same orders was added by accident t...,15-12-2020 04:02,double charged order,two orders added accident restaurant tried rem...
...,...,...,...,...,...,...
485,uber,5,perdí mi cuenta no la puedo recuperar la use e...,16-01-2021 02:39,,perdí mi cuenta la puedo recuperar la use el m...
486,crap crap crap,1,still the same i was forced to use it in colom...,23-12-2020 00:15,crap crap crap,still forced use colombia since know ride shar...
487,sleeping drivers,1,it is a 30 minute commute from my household to...,16-12-2020 19:10,sleeping drivers,minute commute household worksite tend use get...
488,bad design re offer code redemption and issue ...,1,was sent a 30 off uber eats i thought about pl...,25-11-2020 23:06,bad design offer code redemption issue resolution,sent eats thought placing order applied code c...


In [93]:
len(uber_reviews['Title_nostop'].unique())
len(uber_reviews['Review_nostop'].unique())

489

In [94]:
uber_reviews.drop('Title', axis=1, inplace=True)
uber_reviews.drop('Review', axis=1, inplace=True)

#### 2. Basic text and sentiment analysis: You could choose to run these to get a feel for the data. What are people in general saying, etc. What is the distribution of valence across the corpus and within documents. Use bigrams, COGs, wordclouds, phrases etc if you want. May help to randomly read through a few reviews to get a feel, as well.

In [95]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# see what kinda output VADER yields on 1 doc first
vs0 = analyzer.polarity_scores(uber_reviews['Review_nostop'].iloc[0]); vs0

{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'compound': 0.2732}

In [96]:
import nltk 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
uber_sents_list = sent_tokenize(uber_reviews['Review_nostop'].iloc[0])
print(uber_sents_list[:5])  # view first five sents

vs0_sent = analyzer.polarity_scores(uber_sents_list[0]); vs0_sent  # aha. works

['half hour tried every service take short trip falsely advertised dropoffs certain times deliver finally downloded lyft immediately got car two minutes away']


[nltk_data] Downloading package punkt to /Users/Dell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'compound': 0.2732}

In [101]:
data = uber_reviews['Review_nostop']
uber_review = data.copy()

doc0_df = []
# define unit func to process one doc
def vader_unit_func(doc0):
    sents_list0 = sent_tokenize(doc0)
    vs_doc0 = []
    sent_df = []
    for i in range(len(sents_list0)):
        vs_sent0 = analyzer.polarity_scores(sents_list0[i])
        vs_doc0.append(vs_sent0)
        sent_ind.append(i)
        
    doc0_df = pd.DataFrame(vs_doc0)
    doc0_df.insert(0, 'sent_index', sent_ind)  # insert sent index
    doc0_df.insert(doc0_df.shape[1], 'sentence', sents_list0)
    return(doc0_df)

%time doc0_df = vader_unit_func(uber_review['0'])
doc0_df

KeyError: '0'

[]

In [102]:
uber_review

0      half hour tried every service take short trip ...
1      eligible offer stop flooding email false infor...
2      consistently inaccurate eats eta food menus of...
3      rides canceled back back still charged account...
4      two orders added accident restaurant tried rem...
                             ...                        
485    perdí mi cuenta la puedo recuperar la use el m...
486    still forced use colombia since know ride shar...
487    minute commute household worksite tend use get...
488    sent eats thought placing order applied code c...
489    first okay things started happening like drive...
Name: Review_nostop, Length: 490, dtype: object

In [58]:
# define wrapper func
def vader_wrap_func(corpus0):
    
    # use ifinstance() to check & convert input to DF
    if isinstance(corpus0, list):
        corpus0 = pd.DataFrame({'text':corpus0})
    
    # define empty DF to concat unit func output to
    vs_df = pd.DataFrame(columns=['doc_index', 'sent_index', 'neg', 'neu', 'pos', 'compound', 'sentence'])    
    
    # apply unit-func to each doc & loop over all docs
    for i1 in range(len(corpus0)):
        doc0 = str(corpus0.text.iloc[i1])
        vs_doc_df = vader_unit_func(doc0)  # applying unit-func
        vs_doc_df.insert(0, 'doc_index', i1)  # inserting doc index
        vs_df = pd.concat([vs_df, vs_doc_df], axis=0)
        
    return(vs_df)

# test-drive wrapper func
%time uber_vs_df = vader_wrap_func(uber_review)    
uber_vs_df

AttributeError: 'Series' object has no attribute 'text'

NameError: name 'uber_vs_df' is not defined

In [ ]:
## Sentiment analysis proves negative interactions since the data is largely complaints against the drivers which involes wai times and other issues

#### 3. Feature construction and extraction: Recall that client wants to know how text features impact overall ratings. So think about what features in text might impact rating. 1. Full DTM? Sentiment scores? Valence shifted sentiment? Your call on whether or not this applies to this context. 2. Topic scores of each document after running a latent topic model (LTM)? Remember to interpret and label topics if you decide to use LTMs. 3. Number of matches with some custom lexicon or wordlist you built? You could build a lexicon or wordlist of terms used by Uber users (e.g. driver, fare, payment etc) which maynot be caught in the sentiment lexicon and use counts of matched tokens as a feature. 4. Any NLP based features? parts of speech? named entities? etc. 5. Any other feature you think may be relevant in this context. For instance, will the Date field offer useful info? For example, do complaints spike on say, weekends versus weekdays? Rush hour versus other times?

In [103]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

## here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation) using regex
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [104]:
# Use above funcs to iterate over the list of synopses to create two vocabularies: one stemmed and one only tokenized. 
totalvocab_stemmed = []
totalvocab_tokenized = []
uber_review_ = uber_review.tolist()

t0 = time.time()
for i in uber_review_:
    
    # doing both toknz & stemming
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    # doing toknz only
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
t1 = time.time()
print(round(t1-t0, 3))    # 0.2 s

0.306


In [71]:
## create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
vocab_frame

,words
half,half
hour,hour
tri,tried
everi,every
uber,uber
...,...
head,head
charg,charge
done,done
use,using


In [106]:
## Tf-idf and document similarity
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=1, # max proportion of docs word is present in
				   max_features=200000,
                                   min_df=0, 
				   stop_words='english',
                                   use_idf=True, 
				   tokenizer=tokenize_and_stem, 
				   ngram_range=(1,5))

# note magic cmd %time
tfidf_matrix = tfidf_vectorizer.fit_transform(uber_review)    # 6.05 secs

print(tfidf_matrix.shape)    # dimns of the tfidf matrix

(490, 45164)


In [107]:
terms = tfidf_vectorizer.get_feature_names()
terms[:20]

/Users/Dell/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10kyear',
 '10kyear liter',
 '10kyear liter everywher',
 '10kyear liter everywher issu',
 '10kyear liter everywher issu can\x92t',
 '10min',
 '10min driver',
 '10min driver drove',
 '10min driver drove stop',
 '10min driver drove stop cancel',
 '10\x97said',
 '10\x97said estim',
 '10\x97said estim await',
 '10\x97said estim await driver',
 '10\x97said estim await driver pick',
 '14th',
 '14th trip',
 '14th trip steal',
 '14th trip steal custom',
 '14th trip steal custom money']

In [108]:
print(type(tfidf_matrix))

tfidf_matrix.todense()[:5,:5]

<class 'scipy.sparse.csr.csr_matrix'>


matrix([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [109]:
terms = tfidf_vectorizer.get_feature_names()
terms[:20]

['10kyear',
 '10kyear liter',
 '10kyear liter everywher',
 '10kyear liter everywher issu',
 '10kyear liter everywher issu can\x92t',
 '10min',
 '10min driver',
 '10min driver drove',
 '10min driver drove stop',
 '10min driver drove stop cancel',
 '10\x97said',
 '10\x97said estim',
 '10\x97said estim await',
 '10\x97said estim await driver',
 '10\x97said estim await driver pick',
 '14th',
 '14th trip',
 '14th trip steal',
 '14th trip steal custom',
 '14th trip steal custom money']

#### Supervised Learning phase: Rubber meets road now. Run a regression (or classification, if you prefer) of review ratings against the text features you have collected in the previous step. Use any regression or classification method you want to. OLS regression is easy to run and interpret, and is hence preferable.

In [110]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [111]:
## setup chunk
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble

import pandas, xgboost, numpy, string  # , textblob !pip install textblob
import csv,re,nltk
import time

In [126]:
# read data in
labels, texts = [], []
for i in uber_review:
    labels.append(i[0])
    texts.append(i[1])

# build panda DF to house the data
trainDF = pandas.DataFrame()
trainDF['text'] = uber_review
trainDF['label'] = uber_reviews['Rating']
trainDF.iloc[:10]

,text,label
0,half hour tried every service take short trip ...,1
1,eligible offer stop flooding email false infor...,2
2,consistently inaccurate eats eta food menus of...,2
3,rides canceled back back still charged account...,1
4,two orders added accident restaurant tried rem...,1
5,even offer promotions one really wants pick ch...,2
6,hate app canceled ride aint even give money b...,1
7,its good times find drivers gps always miss t...,3
8,customer support cant call representative hor...,1
9,makes difficult make account seemingly impossi...,1


In [127]:
# split the DF into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable into 0/1
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [128]:
# create a count vectorizer TF-DTM object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
t1 = time.time()
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)  # training or calibration sample
xvalid_count =  count_vect.transform(valid_x)  # validation or test sample
t2 = time.time()

print(round(t2 - t1,3), "secs")  # ~ 0.15 secs to create DTM. Not bad, eh?
print(xtrain_count.shape)
print(xvalid_count.shape)

0.05 secs
(367, 3323)
(123, 3323)


In [129]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
t1 = time.time()
tfidf_vect = tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
t2 = time.time()

print(round(t2 - t1,3), "secs")  # ~ 0.15 secs again
print(xtrain_tfidf.shape)
print(xvalid_tfidf.shape)

0.055 secs
(367, 3323)
(123, 3323)


In [130]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    #print(len(predictions))
    predDF = pandas.DataFrame()
    predDF['text'] = valid_x
    predDF['actual_label'] = valid_y
    predDF['model_label'] = predictions
    
    print(predDF.iloc[:8,])
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [131]:
# Naive Bayes on DTM
t1 = time.time()
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
t2 = time.time()
print("\n\n", round(t2-t1,3), "secs for NB on TF\n\n")  # 0.01 secs. Fast!
print("\nNaive Bayes on DTM accuracy: "+ str(accuracy))
print("\n====================\n")

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("\nNaive Bayes on WordLevel TF-IDF accuracy: "+ str(accuracy))

                                                  text  actual_label  \
397  customer service tried use voucher ride app wo...             0   
71   easy inexpensive get held traffic charge pleas...             0   
198  guys need customer service phone line contact ...             0   
185  main app great integrated ubereats many featur...             0   
348  honestly sick tired support agents dont jobs ...             0   
427  put 60 card added gift card account go request...             0   
308  plain simple never get lyft possible worse eve...             0   
450  multiple times ive called wait time one unrea...             1   

     model_label  
397            0  
71             0  
198            0  
185            0  
348            0  
427            0  
308            0  
450            0  


 0.01 secs for NB on TF



Naive Bayes on DTM accuracy: 0.6260162601626016


                                                  text  actual_label  \
397  customer service tried u

#### Q5. Recommendations based on the analysis

In [ ]:
# based on the study conducted there is a 69.9% corelation between the ratings given by the passenger and the review given.
# The rating is given in the labels to access the relationship between the rating and the comments given by the passengers
# Largely the complaints have been around waittimings, customer service and driver related issues
